# Embedding Watermark

### Importing necessary libraries

In [1]:
from PIL import Image
import numpy as np
import cv2

### IMPORTING image from local storage

In [2]:
# Load the host and watermark images
hr_image = Image.open("host_image.jpg")
wm_image = Image.open("./watermark_image.jpg")

### Resizing the watermarked image in size of host image

In [3]:
# Resize the watermark image to the same size as the host image
wm_image_resized = wm_image.resize(hr_image.size)

# Convert both images to grayscale
hr_image = hr_image .convert("L")
wm_image_resized = wm_image_resized.convert("L")


### Function to apply SVD and modify singular values

In [4]:

def embed_watermark(hr_channel, wm_channel, alpha=0.6):
    U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel)
    U_wm, sigma_wm, V_wm = np.linalg.svd(wm_channel)
    
    sigma_hr_mod = sigma_hr + alpha * sigma_wm
    hr_channel_mod = np.dot(U_hr[:, :sigma_hr_mod.shape[0]], np.dot(np.diag(sigma_hr_mod), V_hr[:sigma_hr_mod.shape[0],:]))
    
    return hr_channel_mod

In [5]:
hr_mod = embed_watermark(hr_image, wm_image_resized)

### Covert the array in range of 0 to 255

In [6]:
hr_mod = cv2.normalize(hr_mod, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
hr_mod

array([[24, 24, 23, ..., 29, 29, 28],
       [24, 23, 23, ..., 29, 28, 28],
       [24, 23, 23, ..., 28, 28, 28],
       ...,
       [ 3,  3,  3, ...,  5,  5,  5],
       [ 3,  3,  3, ...,  5,  5,  5],
       [ 3,  3,  3, ...,  5,  5,  5]], dtype=uint8)

### Extract the image from the array

In [7]:
watermarked_image_grayscale = Image.fromarray(hr_mod)
watermarked_image_grayscale.show()
watermarked_image_grayscale.save('output_image.png')

# Extraction 

In [8]:
# Extraction process
def extract_watermark(hr_mod_channel, hr_channel,wr_channel, alpha=0.6):
    U_hr_mod, sigma_hr_mod, V_hr_mod = np.linalg.svd(hr_mod_channel, full_matrices=False)
    U_hr, sigma_hr, V_hr = np.linalg.svd(hr_channel, full_matrices=False)
    U_wm, sigma_wm, V_wm = np.linalg.svd(wr_channel, full_matrices=False)
    
    sigma_wm_mod = (sigma_hr_mod - sigma_hr) / alpha
    wm_channel_mod = np.dot(U_wm[:, :sigma_wm_mod.shape[0]], np.dot(np.diag(sigma_wm_mod), V_wm[:sigma_wm_mod.shape[0],:]))
    return wm_channel_mod

In [9]:
wm_mod = extract_watermark(hr_mod, hr_image,wm_image_resized)

In [10]:
wm_mod = cv2.normalize(wm_mod, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)

### Inverse SVD to obtain the Watermark Image

In [11]:
extracted_watermark=Image.fromarray(wm_mod)
extracted_watermark.show()
extracted_watermark.save('watermark_output_image.png')